In [7]:
%matplotlib inline
from PIL import Image
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Conv2D, MaxPooling2D
#from keras import backend as K

path1 = "/Users/gene/Downloads/dog.jpg"
path2 = "/Users/gene/Learn/texture_nets/data/readme_pics/kitty.jpg"

In [32]:
def make_model():
    model = Sequential()
    model.add(Dense(2, kernel_initializer="normal", activation='relu', input_shape=(2,)))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(20, kernel_initializer="normal", activation='relu'))
    model.add(Dense(3, kernel_initializer="normal"))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

m1 = make_model()
m2 = make_model()


In [8]:
def get_data(img):
    pixels = img.getdata()
    x_data, y_data = [],[]
    for y in range(height):
        for x in range(width):
            idx = x + y * width
            r, g, b = pixels[idx]
            x_data.append([x/float(width), y/float(height)])
            y_data.append([r, g, b])
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    return x_data, y_data

im1 = Image.open(path1)
im2 = Image.open(path2)
im2 = im2.resize(im1.size)
width, height = im1.size

x1, y1 = get_data(im1)
x2, y2 = get_data(im2)


In [9]:
m1.fit(x1, y1, batch_size=64, epochs=20, verbose=1, validation_data=(x1, y1))
m2.fit(x2, y2, batch_size=64, epochs=20, verbose=1, validation_data=(x2, y2))


Train on 90000 samples, validate on 90000 samples
Epoch 1/20
90000/90000 [==============================] - 0s - loss: 5175.5671 - val_loss: 3205.5965
Epoch 2/20
90000/90000 [==============================] - 0s - loss: 3179.6136 - val_loss: 3170.7304
Epoch 3/20
90000/90000 [==============================] - 0s - loss: 3171.0338 - val_loss: 3167.3256
Epoch 4/20
90000/90000 [==============================] - 0s - loss: 3169.3200 - val_loss: 3166.5097
Epoch 5/20
90000/90000 [==============================] - 0s - loss: 3169.0254 - val_loss: 3166.5482
Epoch 6/20
90000/90000 [==============================] - 0s - loss: 3168.5353 - val_loss: 3175.4782
Epoch 7/20
90000/90000 [==============================] - 0s - loss: 3168.4852 - val_loss: 3169.8046
Epoch 8/20
90000/90000 [==============================] - 0s - loss: 3168.1716 - val_loss: 3171.5619
Epoch 9/20
90000/90000 [==============================] - 0s - loss: 3165.8357 - val_loss: 3163.1344
Epoch 10/20
90000/90000 [================

In [28]:
def get_interpolated_image(model1, model2, amt):
    w1 = np.array(model1.get_weights())
    w2 = np.array(model2.get_weights())
    w3 = np.add((1.0 - amt) * w1, amt * w2)
    model.set_weights(w3)
    img = Image.new("RGB", [width, height])
    pixels = img.load()
    y_pred = model.predict(x1)
    print(y_pred)
    for y in range(height):
        for x in range(width):
            idx = x + y * width
            r, g, b = y_pred[idx]
            pixels[x, y] = (int(r), int(g), int(b))
#     img.save("/Users/gene/Downloads/mytest.png", "png")
    return img
    #im_new.save(path, "png")


#for i in range(10):
#  do_interp("/Users/gene/Downloads/dog2_3_%d.png"%i, float(i)/(9))


# print width, height

In [31]:

im = get_interpolated_image(m1, m2, 0.0)
im.show()

[[  59.40284348   68.16860199   37.71345901]
 [  59.5620842    68.31572723   37.82919693]
 [  59.72131729   68.46286774   37.94493103]
 ..., 
 [ 203.40966797  201.23262024  142.37908936]
 [ 203.56889343  201.37974548  142.49481201]
 [ 203.72813416  201.52688599  142.61056519]]


In [ ]:

im2 = Image.new("RGB", [width, height])
pixels = im2.load()
y_pred = model1.predict(x1)
for y in range(height):
  for x in range(width):
    idx = x + y * width
    r, g, b = y_pred[idx] * 255
    pixels[x, y] = (int(r), int(g), int(b))

im2.save("/Users/gene/Downloads/dog2_33.png", "png")



w1 = np.array(model1.get_weights())
w2 = np.array(model2.get_weights())

w3 = np.add(w1, w2)


